# Binary classification

In [1]:
import numpy as np
import numpy as np
# from ecog_band.models import SVMBinClassifier
# from ecog_band.models import SVMBinClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import os
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
# from ecog_band.utils import *
# from Ecog_pretrain.ecog_band.utils import *
# from Ecog_pretrain.ecog_band.solver import Nfold_solver
import pandas as pd
# from ecog_band.datasetExcludeBand import CustomDatasetExcband
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
import torch
from torch.utils import data as Data
from sklearn.model_selection import GridSearchCV,train_test_split
import torch.nn as nn
import seaborn as sns
from matplotlib.colors import Normalize
from matplotlib.patches import Rectangle
import scipy.io as scio
import matplotlib.image as mpimg
import matplotlib as mpl
import matplotlib.patches as patches
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


bands_list = ['theta', 'alpha', 'beta1', 'beta2', 'low gamma', 'high gamma']
freq_list=[400]
HS_list=[68, 69, 75, 79, 82, 83, 84, 85, 86]
mpl.rcParams['font.size'] =10

In [2]:
class BandErpDataset(Data.Dataset):
    def __init__(self, band_covert_data_all, band_overt_data_all, elec, band):
        '''band_covert_data_all, z_band_read_data_all为所有frequency bands的数据字典，每个band对应一个任务的所有trail的数据
        band_covert_data_all = {band : data(shape = n_trails, n_electrodes, n_timepoints)}
        '''
        super().__init__()
        self.data = [] # 存Ecog数据，shape为（n_samples x n_features)
        self.labels = [] # 存label, shape=（n_samples），cue标记为0，read标记为1
        # load aligned data and plot average signal
        cue_data = band_covert_data_all[band]
        read_data = band_overt_data_all[band] 

        X=np.concatenate((cue_data[:, elec, :], read_data[:, elec, :]),axis=0)
        # X = X.reshape(-1, X.shape[1]*X.shape[2])

        y_cue=np.array([0]*cue_data.shape[0])
        y_read=np.array([1]*read_data.shape[0])
        y=np.concatenate((y_cue,y_read),axis=0)

        self.data = X
        self.labels = y

    def __len__(self):
        return len(self.data)
    
    def get_data_labels(self):
        return self.data, self.labels

    def __getitem__(self, idx):
        # Return a tuple of (data, label) and convert data to a PyTorch tensor
        # print(self.data.shape)
        return torch.tensor(abs(self.data[idx]), dtype=torch.float), torch.tensor(self.labels[idx])
    
class SVMBinClassifier(nn.Module):
    def __init__(self):
        # tuned_parameters = [{'kernel': ['linear'], 'C': [0.1, 1], 'gamma': [0.01]}]
        # self.model = GridSearchCV(SVC(), tuned_parameters, scoring='accuracy', n_jobs=1)
        self.model = SVC(C=0.1, kernel='sigmoid')
        self.pca = PCA(n_components=0.95)
    
    def train(self, x_train, y_train):
        X_train_pca = self.pca.fit_transform(x_train)
        self.model.fit(X_train_pca, y_train)
        # plt_learning_curve(self.model, X_train_pca, y_train, cv=5)
        # best_params = self.model.best_params_
        # return best_params
    
    def predict(self, x_test):
        X_test_pca = self.pca.transform(x_test)
        return self.model.predict(X_test_pca)
    
    def evaluate(self, X_test, y_test):
        # X_test_pca = self.pca.transform(X_test)
        y_pred = self.predict(X_test)
        # y_pred = cross_val_predict(self.model, X=X_test_pca, y=y_test, cv=5)
        return y_pred

In [8]:
def cal_bc_sig_HS_band(HS, freq, bands_list, elec_list, band_cue_data_all, band_read_data_all):
    '''
    discription: 计算所有band在256个电极上的accuracy
    band_cue_data_all，band_read_data_all是所有band的数据字典（z-score之后的）
    return：bands_acc 字典，每个band的acc是一个长度为256的list {band: [accList], band:[], ...}
    新增：返回每个任务下的分类准确率bands_acc_read， bands_acc_cue
    '''
    bands_acc = {} # 存储每个band的分类准确率
    bands_acc_cue = {}  # 新增：存储每个band在cue任务下的分类准确率
    bands_acc_read = {}  # 新增：存储每个band在read任务下的分类准确率
    for band in bands_list:
        band_elec_acc = []
        band_elec_acc_cue = []  # 新增：存储每个电极在cue任务下的准确率
        band_elec_acc_read = []  # 新增：存储每个电极在read任务下的准确率
        for i, elec in enumerate(elec_list):
            data_loader = BandErpDataset(band_cue_data_all, band_read_data_all, elec, band)
            data_all, labels_all = data_loader.get_data_labels()
            # print(f'data_shape: {data_all.shape}, data_label_shape:{labels_all.shape}')

            X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=1/6, random_state=42, shuffle=True)
            # svm = DecisionTreeBinClassifier()
            svm = SVMBinClassifier()
            svm.train(x_train=X_train, y_train=y_train)

            y_pred = svm.evaluate(X_test=X_test, y_test=y_test)
            elec_acc = accuracy_score(y_test, y_pred)
            band_elec_acc.append(elec_acc)
            
            # 计算 cue 和 read 的分类准确率
            cue_indices = np.where(y_test == 0)[0]  # cue 的标签为 0
            read_indices = np.where(y_test == 1)[0]  # read 的标签为 1
            
            if len(cue_indices) > 0:  # 确保cue有数据
                elec_acc_cue = accuracy_score(y_test[cue_indices], y_pred[cue_indices])
                band_elec_acc_cue.append(elec_acc_cue)
            
            if len(read_indices) > 0:  # 确保read有数据
                elec_acc_read = accuracy_score(y_test[read_indices], y_pred[read_indices])
                band_elec_acc_read.append(elec_acc_read)
        
        bands_acc[band] = band_elec_acc
        bands_acc_cue[band] = band_elec_acc_cue  # 新增：存储每个band在cue任务下的分类准确率
        bands_acc_read[band] = band_elec_acc_read  # 新增：存储每个band在read任务下的分类准确率
        
        print(f'Accuracy for HS{HS}-freq{freq} on {band} band is Done! acc_list={band_elec_acc}')
    
    return bands_acc, bands_acc_cue, bands_acc_read
            


In [ ]:
elec_list = [i for i in range(0, 256)]
save_acc_dir = f'/public/DATA/overt_reading/bands_acc/'
os.makedirs(save_acc_dir, exist_ok=True)

for HS in HS_list:
    for freq in freq_list:
        data_path = f'/public/DATA/overt_reading/aligned_data/HS{HS}'
        save_acc_path = os.path.join(save_acc_dir, f'HS{HS}', f'{freq}')
        os.makedirs(save_acc_path, exist_ok=True)
        cue_data = np.load(os.path.join(data_path, f'cue_allbands_{freq}hz.npy'), allow_pickle=True).item()
        read_data = np.load(os.path.join(data_path, f'read_allbands_{freq}hz.npy'), allow_pickle=True).item()
        bands_acc, bands_acc_cue, bands_acc_read = cal_bc_sig_HS_band(HS, freq, bands_list, elec_list, cue_data, read_data)
        np.save(os.path.join(save_acc_path, 'bands_acc.npy'), bands_acc)
        np.save(os.path.join(save_acc_path, 'bands_acc_cue.npy'), bands_acc_cue)
        np.save(os.path.join(save_acc_path, 'bands_acc_read.npy'), bands_acc_read)


Accuracy for HS68-freq400 on theta band is Done! acc_list=[0.4519230769230769, 0.4326923076923077, 0.5673076923076923, 0.6153846153846154, 0.4519230769230769, 0.4423076923076923, 0.4423076923076923, 0.4807692307692308, 0.47115384615384615, 0.5288461538461539, 0.4519230769230769, 0.4807692307692308, 0.5384615384615384, 0.47115384615384615, 0.46153846153846156, 0.49038461538461536, 0.5288461538461539, 0.4423076923076923, 0.4423076923076923, 0.4423076923076923, 0.6346153846153846, 0.4423076923076923, 0.4423076923076923, 0.47115384615384615, 0.46153846153846156, 0.5288461538461539, 0.4423076923076923, 0.4519230769230769, 0.5096153846153846, 0.47115384615384615, 0.5192307692307693, 0.4807692307692308, 0.5576923076923077, 0.5, 0.4423076923076923, 0.4326923076923077, 0.47115384615384615, 0.5865384615384616, 0.4230769230769231, 0.47115384615384615, 0.47115384615384615, 0.5288461538461539, 0.4423076923076923, 0.47115384615384615, 0.5, 0.47115384615384615, 0.5, 0.5096153846153846, 0.471153846153